In [1]:
def solve_komo_IK_grasping():
    IK = C.komo_IK(False)

    # encode grasping
    IK.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["object", "R_gripperCenter"], 
                    ry.OT.eq, 
                    1e2,
                    [0.,0.,.0]);

    IK.addObjective([1.], 
                    ry.FS.quaternionRel, 
                    ["object", "R_gripperCenter"], 
                    ry.OT.eq, 
                    [1e2],
                    [.707,.707,0,0]);


    IK.optimize()
#     IK.getReport()
    return IK.getPathFrames()  

def solve_komo_IK_grasping_relative():
    IK = C.komo_IK(False)

    # encode grasping
    IK.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["object", "R_gripperCenter"], 
                    ry.OT.eq, 
                    1e2,
                    [0.,0.,.0]);

    IK.addObjective([0.7, 1.], 
                    ry.FS.scalarProductXZ, 
                    ["R_gripperCenter", "object"], 
                    ry.OT.eq, 
                    [1e2],
                    [0.])

    IK.addObjective([0.7, 1.], 
                    ry.FS.scalarProductZ, 
                    ["R_gripperCenter", "world"], 
                    ry.OT.eq, 
                    [1e2],
                    [0.])

    IK.optimize()
#     IK.getReport()
    return IK.getPathFrames()  


In [2]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry
import time
import cv2 as cv

#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")

#you can also change the shape & size
RealWorld.getFrame("obj0").setColor([1.,0,0])
RealWorld.getFrame("obj0").setShape(ry.ST.box, [.04, .04, .15, .01])
# RealWorld.getFrame("obj0").setPosition([0., .2, 1.])
RealWorld.getFrame("obj0").setPosition([0., 0., .68])
RealWorld.getFrame("obj0").setQuaternion([0.707, 0.707, 0., 0.])

RealWorld.getFrame("obj0").setContact(1)

#remove some objects
for o in range(1,30):
    name = "obj%i" % o
#     print("deleting", name)
    RealWorld.delFrame(name)
    
    
# instantiate the simulation
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera")

# we're adding an "imp" to the simulation, which is a little process that can inject perturbations
# S.addImp(ry.ImpType.objectImpulses, ['obj0'], [])

# setup your model world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")

obj = C.addFrame("object")
obj.setColor([1.,0,0]) # set the color of one objet to red!
obj.setShape(ry.ST.sphere, [.03]) # assume that we know the object shape

# obj.setQuaternion([1,0,1,0])
# obj.setShape(ry.ST.capsule, [.2,.02])
# obj.setColor([1,0,1])

D = C.view();

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/05-grasp'

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}


** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback
** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/05-grasp'
** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}



In [3]:
def solve_komo_IK_grasping():
    IK = C.komo_IK(False)

    # encode grasping
    IK.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["object", "R_gripperCenter"], 
                    ry.OT.eq, 
                    1e2,
                    [0.,0.,.0]);


    IK.addObjective([0.,1.], 
                    ry.FS.quaternionRel, 
                    ["object", "R_gripperCenter"], 
                    ry.OT.eq, 
                    [1.2e2],
                    [0.707, 0.707, 0., 0.]);


    IK.optimize()
#     IK.getReport()
    return IK.getPathFrames()  


In [4]:
def grasping(KOMO = False):
    points = []
    tau = .01

    gripping = False
    grasped = False

    prev_contours = None
    prev_full_mask = None
    # small cheating, better intitialization
    objectPosition = RealWorld.frame("obj0").getPosition();

    for t in range(3000):
        time.sleep(tau)

        #grab sensor readings from the simulation
        q = S.get_q()

        # TOTAL CHEAT: grab the true position from the RealWorld
        objectPosition = RealWorld.frame("obj0").getPosition();
        objectRotation = RealWorld.frame("obj0").getQuaternion();
        obj.setPosition(objectPosition) # set the model object to percept
        obj.setQuaternion(objectRotation) # set the model object to percept
        
        C.setJointState(q); # set your robot model to match the real q

        y, J = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "object"])

    #     qdot = np.linalg.inv(J.T@J + 1e-3*np.eye(q.shape[0])) @ J.T @ (-5*y)

        komo_next_state = solve_komo_IK_grasping()[0]
#         komo_next_state = solve_komo_IK_grasping_relative()[0]
            
        C.setFrameState(komo_next_state)

        qdot = ( C.getJointState() - q ) /  (  40*tau )

        if not gripping and np.linalg.norm(y) < .01:
            print("Grasping")
            S.closeGripper("R_gripper", speed=1.)
            gripping = True

        if gripping and S.getGripperWidth("R_gripper") < 1e-2:
            print("FAILED!")
            S.openGripper("R_gripper", speed=1.)
            gripping = False
            return False

        if gripping and S.getGripperIsGrasping("R_gripper"):
            print("GRASPED!")
            grasped = True
            return True

        S.step(qdot, tau, ry.ControlMode.velocity)
    


In [5]:
# while (True):
#     grasped = grasping()
#     if not grasped:
#         print("Failiure grasping")
#         break;
#     released = lift_release()
#     if not released:
#         print("Failiure releasing")
#         break;

In [ ]:
grasped = grasping()

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.00125 (kin:7.6e-05 coll:0 feat:0.000597 newton: 0.000144) setJointStateCount:29
   sos:0.0316569 ineq:0 eq:0.0260366
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.001073 (kin:6.9e-05 coll:0 feat:0.000483 newton: 0.000117) setJointStateCount:26
   sos:0.0301747 ineq:0 eq:0.0261486
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.001412 (kin:7.2e-05 coll:0 feat:0.00049 newton: 0.000429) setJointStateCount:25
   sos:0.0287481 ineq:0 eq:0.0261336
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.001013 (kin:6.2e-05 coll:0 feat:0.000452 new

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.001012 (kin:6.4e-05 coll:0 feat:0.000397 newton: 8.5e-05) setJointStateCount:17
   sos:0.00643947 ineq:0 eq:0.0259134
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000725 (kin:4.4e-05 coll:0 feat:0.000296 newton: 7.6e-05) setJointStateCount:16
   sos:0.00612115 ineq:0 eq:0.0260561
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.00089 (kin:4.9e-05 coll:0 feat:0.000405 newton: 0.000112) setJointStateCount:17
   sos:0.00580788 ineq:0 eq:0.025897
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000668 (kin:3.8e-05 coll:0 feat:0.000284 ne

** optimization time:0.000961 (kin:0.000102 coll:0 feat:0.000243 newton: 5.7e-05) setJointStateCount:10
   sos:0.000998922 ineq:0 eq:0.0287189
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000643 (kin:4.1e-05 coll:0 feat:0.000269 newton: 5.9e-05) setJointStateCount:10
   sos:0.000956643 ineq:0 eq:0.0286593
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000602 (kin:3.5e-05 coll:0 feat:0.000256 newton: 5.8e-05) setJointStateCount:10
   sos:0.000903625 ineq:0 eq:0.0283088
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000639 (kin:3.4e-05 coll:0 feat:0.000259 newton: 6.3e-05) setJointStateCount:10
   sos:0.000865025 ineq:0 eq:0.0286806
** KOMO::run solver:dense collisions:0 x-dim:14

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000473 (kin:2.5e-05 coll:0 feat:0.000173 newton: 4.3e-05) setJointStateCount:7
   sos:0.000158916 ineq:0 eq:0.0269017
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000808 (kin:2.5e-05 coll:0 feat:0.000177 newton: 0.000384) setJointStateCount:7
   sos:0.000147382 ineq:0 eq:0.026784
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000459 (kin:2.3e-05 coll:0 feat:0.00017 newton: 4.5e-05) setJointStateCount:7
   sos:0.000145869 ineq:0 eq:0.0264993
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000432 (kin:2.4e-05 coll:0 feat:0.000166 ne

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.00095 (kin:2.7e-05 coll:0 feat:0.000157 newton: 0.000481) setJointStateCount:5
   sos:2.87094e-05 ineq:0 eq:0.0265995
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000243 (kin:1e-05 coll:0 feat:8.3e-05 newton: 2.2e-05) setJointStateCount:3
   sos:3.28599e-05 ineq:0 eq:0.0332587
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000367 (kin:1.8e-05 coll:0 feat:0.000131 newton: 3.6e-05) setJointStateCount:5
   sos:4.48127e-05 ineq:0 eq:0.0261944
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000485 (kin:3e-05 coll:0 feat:0.000157 newton

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000292 (kin:1.3e-05 coll:0 feat:9.6e-05 newton: 2.3e-05) setJointStateCount:3
   sos:2.28245e-05 ineq:0 eq:0.0284477
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000295 (kin:1.3e-05 coll:0 feat:0.0001 newton: 2.3e-05) setJointStateCount:3
   sos:1.10751e-05 ineq:0 eq:0.032914
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000764 (kin:0.000107 coll:0 feat:0.000474 newton: 2.2e-05) setJointStateCount:3
   sos:9.59596e-06 ineq:0 eq:0.0285881
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000704 (kin:2.8e-05 coll:0 feat:0.000265 newt